In [ ]:
import msal
from msal import PublicClientApplication
import webbrowser
import requests
import os
import requests
#from ms_graph import generate_access_token, GRAPH_API_ENDPOINT

In [ ]:
## criar uma API no Azure e configura-lá
## na URI de redirecionamento com o link gerado copiar o código de autorização
APPLICATION_ID = "Aplicação cliente Azure"
CLIENT_SECRET = "chave secreta da aplicação Azure"
authority_url = "https://login.microsoftonline.com/consumers/"
base_url = "https://graph.microsoft.com/v1.0/"

endpoint = base_url + 'me'
SCOPES = ['Permissões APIs','Permissões APIs']

# metodo 1: autenticação com codigo de autorização

client_instance =  msal.ConfidentialClientApplication(
    client_id = APPLICATION_ID,
    client_credential=CLIENT_SECRET,
    authority= authority_url
)

authorization_request_url = client_instance.get_authorization_request_url(SCOPES)
print(authorization_request_url)
webbrowser.open(authorization_request_url, new=True)




https://login.microsoftonline.com/consumers/oauth2/v2.0/authorize?client_id=fec498f0-ddbb-42fb-8b21-fa02cb7210ff&response_type=code&scope=Files.ReadWrite.All+User.Read+offline_access+openid+profile


False

In [ ]:
## colar o código de autorização aqui provavelmente única parte manual
authorization_code = 'Código presente na sua URI de redirecionamento'

In [ ]:
## com o código de autorização podemos gerar o token para termos permissão para acessar as aplicações microsoft através das APIs azure
access_token = client_instance.acquire_token_by_authorization_code(
     code = authorization_code,
     scopes = SCOPES)

access_token_id = access_token['access_token']
headers = {'Authorization': 'Bearer ' + access_token_id}

In [ ]:
## nessa etapa forneceremos o id do arquivo que queremos baixar e baixamos o arquivo
save_location = os.getcwd()

file_ids = ['ID de seu arquivo']

headers = {
	'Authorization': 'Bearer ' + access_token['access_token']
}

# Step 1.  obtenha o nome do arquivo
for file_id in file_ids:
	response_file_info = requests.get(
		base_url + f'/me/drive/items/{file_id}',
		headers=headers,
		params={'select': 'name'}
	)
	file_name = response_file_info.json().get('name')

	# Step 2. baixando o arquivo OneDrive
	response_file_content = requests.get(base_url + f'/me/drive/items/{file_id}/content', headers=headers)
	with open(os.path.join(save_location, file_name), 'wb') as _f:
		_f.write(response_file_content.content)